In [1]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
from diffusers.utils import load_image, make_image_grid
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionXLImg2ImgPipeline
from datasets import load_dataset
import os
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import lpips
from pathlib import Path
import cv2
from math import sqrt
import inspect, diffusers
from diffusers import AutoencoderKL, DDPMScheduler, StableDiffusionInstructPix2PixPipeline, UNet2DConditionModel

E:\anaconda\envs\diffusers_5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "E:\anaconda\envs\diffusers_5\lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
ModuleNotFoundError: No module named 'triton'


In [ ]:
"""
create img
"""

In [77]:
dataset = load_dataset("imagefolder", data_dir="E:/RainTest/Rain100H_512/input")

In [5]:
model_id = "instruct-pix2pix-model_base_23299"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.bfloat16 ,safety_checker=None).to("cuda")
generator = torch.Generator("cuda").manual_seed(42)

Loading pipeline components...: 100%|██████████| 6/6 [00:03<00:00,  1.92it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_instruct_pix2pix.StableDiffusionInstructPix2PixPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [93]:
prompt = "derain the image"
num_inference_steps = 60
image_guidance_scale = 1.03
guidance_scale = 2
uncond_scale=0.01
mode = "1"
output_dir = "E:/RainTest/output/Rain100H_mode1"



In [100]:
for example in dataset['train']:
    image = example["image"] 
    image = image.convert("RGB") 
    
    original_filename = os.path.basename(example["image"].filename)

    image_path = os.path.join(output_dir, original_filename)  

    edited_image = pipe(prompt,
        image=image,
        mode=mode,
        uncond_scale=uncond_scale,
        num_inference_steps=num_inference_steps,
        image_guidance_scale=image_guidance_scale,
        guidance_scale=guidance_scale,
        generator=generator,
    ).images[0]
    edited_image.save(image_path, "PNG")

    

100%|██████████| 60/60 [00:04<00:00, 12.87it/s]


In [179]:
"""
score
"""

'\nscore\n'

In [10]:
def compute_ssim(img1, img2):
    """
    range: [0, 1]
    return : ssim
    """
    return structural_similarity(img1, img2, data_range=255, channel_axis=-1)
    
def compute_lpips(img1, img2):
    """
    img1, img2: NumPy array，shape = (H, W, 3)，
    range: [0, 255].
    return:lpips
    """
    #to tensor, shape:(N, C, H, W)
    t1 = torch.from_numpy(img1).permute(2, 0, 1).unsqueeze(0).float()
    t2 = torch.from_numpy(img2).permute(2, 0, 1).unsqueeze(0).float()
    # [-1, 1]
    t1 = t1 / 255.0 * 2 - 1
    t2 = t2 / 255.0 * 2 - 1

    with torch.no_grad():
        dist = lpips_fn(t1, t2)
    return dist.item()

def compute_psnr(img1, img2):
    """
    img1, img2: NumPy array，shape = (H, W, 3)
    range: [0, 255]
    return: psnr
    """
    return peak_signal_noise_ratio(img1, img2, data_range=255)

def evaluate_metrics(dataset_input, dataset_gt):
    psnr_values = []
    ssim_values = []
    lpips_values = []

    n_data = len(dataset_input["train"])
    print("All:",n_data)

    for idx in range(n_data):

        input_image = dataset_input["train"][idx]["image"]  
        gt_image = dataset_gt["train"][idx]["image"]       

        input_np = np.array(input_image)
        gt_np = np.array(gt_image)

        #PSNR
        psnr_val = compute_psnr(gt_np, input_np)
        psnr_values.append(psnr_val)

        #SSIM
        ssim_val = compute_ssim(gt_np, input_np)
        ssim_values.append(ssim_val)

        #LPIPS
        lpips_val = compute_lpips(gt_np, input_np)
        lpips_values.append(lpips_val)


    #mean
    mean_psnr = np.mean(psnr_values)
    mean_ssim = np.mean(ssim_values)
    mean_lpips = np.mean(lpips_values)

    return mean_psnr, mean_ssim, mean_lpips

In [101]:
dataset_gt = load_dataset(
    "imagefolder",
    data_dir="E:/RainTest/Rain100H_512/target"
)

dataset_edited = load_dataset(
    "imagefolder",
    data_dir="E:/RainTest/output/Rain100H_mode1"
)

Generating train split: 100 examples [00:00, 28501.66 examples/s]


In [ ]:
def get_filenames(ds):
    return [os.path.basename(ex["image"].filename) for ex in ds["train"]]

names_before = get_filenames(dataset_edited)
names_after  = get_filenames(dataset_gt)
if names_before == names_after:
    print("一致")

In [ ]:
lpips_fn = lpips.LPIPS(net='alex')

mean_psnr_base, mean_ssim_base, mean_lpips_base = evaluate_metrics(dataset_edited, dataset_gt)
print(f"base平均 PSNR:  {mean_psnr_base}")
print(f"base平均 SSIM:  {mean_ssim_base:}")
print(f"base平均 LPIPS: {mean_lpips_base}")